## Traditional Physics-Informed Neural Networks for 1D Singularly Perturbed Problems

### Environment Setup

This cell imports core libraries (NumPy, PyTorch, Matplotlib), selects `DEVICE` (GPU if available, else CPU), and sets the global default dtype to 64-bit floating point for higher precision.
We also set random seeds for reproducibility.

In [ ]:
# ==== Imports & dtype/device ====
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.set_default_dtype(torch.float64)

SEED = 1234
torch.manual_seed(SEED); np.random.seed(SEED)


### Model: Simple MLP

We define a vanilla fully connected neural network to approximate the solution $u_\theta(x)$. 
The network stacks `depth` Tanh-activated layers of `width` neurons and outputs a scalar.
Tanh is smooth and well-suited for approximating functions that require differentiability up to $u''(x)$.

In [ ]:
# ==== Simple MLP (vanilla fully connected network)====
class MLP(nn.Module):
    def __init__(self, in_dim=1, out_dim=1, width=64, depth=4):
        super().__init__()
        layers = [nn.Linear(in_dim, width), nn.Tanh()]
        for _ in range(depth - 2):
            layers += [nn.Linear(width, width), nn.Tanh()]
        layers += [nn.Linear(width, out_dim)]
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)


### Physics Residual

Given a callable $u(x)$ (which can be the raw network output or a hard-encoded trial function), this computes the residual of the 1D singularly perturbed ODE
$$ -\varepsilon\,u''(x) + b(x)\,u'(x) + c(x)\,u(x) - f(x) = 0. $$

We use `torch.autograd` to obtain $u'(x)$ and $u''(x)$ with `create_graph=True` so the residual can be differentiated during training.

In [ ]:
# ==== PDE residual (residual)====
# u_fn: callable, u_fn(x) -> u(x); 这里兼容hard-encoding后的试函数or原始网络输出
def pde_residual(u_fn, x, eps):
    x.requires_grad_(True)
    u  = u_fn(x)
    du = torch.autograd.grad(u, x, torch.ones_like(u), create_graph=True)[0]
    d2u= torch.autograd.grad(du, x, torch.ones_like(du), create_graph=True)[0]
    res = -eps * d2u + b_fun(x) * du + c_fun(x) * u - f_fun(x)
    return res


### Boundary Loss (Soft Constraints)

This section (soft BCs) defines the penalty terms for boundary conditions.
It is designed to support both Dirichlet $u(0)=\alpha$, $u(1)=\beta$ and Neumann $u'(0)=q_0$, $u'(1)=q_1$ conditions, adding them to the loss with suitable weights.

In [ ]:
# ==== Boundary loss (Boundary loss (soft constraint))====
# Supports Dirichlet/Neumann boundary types
def boundary_loss(u_fn, nedge, bc_left, bc_right):
    # 左端 x=0
    x0 = torch.zeros(nedge//2, 1, device=DEVICE, dtype=torch.float64)
    x0.requires_grad_(True)
    u0  = u_fn(x0)
    du0 = torch.autograd.grad(u0, x0, torch.ones_like(u0), create_graph=True)[0]

    # 右端 x=1
    x1 = torch.ones(nedge - nedge//2, 1, device=DEVICE, dtype=torch.float64)
    x1.requires_grad_(True)
    u1  = u_fn(x1)
    du1 = torch.autograd.grad(u1, x1, torch.ones_like(u1), create_graph=True)[0]

    def one_side_loss(kind_val, u, du):
        kind, val = kind_val
        kind = kind.lower()
        if kind == 'dirichlet':
            return torch.mean((u - float(val))**2)
        elif kind == 'neumann':
            return torch.mean((du - float(val))**2)
        else:
            raise ValueError("BC must be 'dirichlet' or 'neumann'.")

    return one_side_loss(bc_left, u0, du0) + one_side_loss(bc_right, u1, du1)


### Hard-encoding the Boundary Conditions

We construct a **trial function** of the form
$$ u(x) = A(x) + S(x)\,N_\theta(x), $$
where:
- $A(x)$ satisfies the boundary conditions exactly.
- $S(x)$ vanishes (value or derivative) at the corresponding end so that adding $S(x)N_\theta(x)$ does not spoil the BCs satisfied by $A(x)$.

The code covers all four combinations of Dirichlet/Neumann at $x=0$ and $x=1$.

In [ ]:
# ==== Hard-encoding (hard-encoding)trial function construction for the four boundary combinations ====
# Template： u(x) = A(x) + S(x) * N(x)
# A(x) satisfies boundary conditions；S(x) vanishes (function or derivative) at the corresponding end without breaking the boundary conditions satisfied by A(x).
def build_hard_u(model, bc_left, bc_right):
    kindL, valL = bc_left[0].lower(), float(bc_left[1])
    kindR, valR = bc_right[0].lower(), float(bc_right[1])

    def A_and_S(x):
        # 方便广播
        # 注意：x 是 (N,1) tensor
        if kindL == 'dirichlet' and kindR == 'dirichlet':
            # A(x) = α(1-x) + βx,  S(x) = x(1-x)
            A = valL*(1.0 - x) + valR*x
            S = x*(1.0 - x)

        elif kindL == 'dirichlet' and kindR == 'neumann':
            # A(x) = α + q1 * x^2/2,  S(x) = x(1-x)^2  (使 S(0)=0, S'(1)=0)
            A = valL + valR * (x**2)/2.0
            S = x*(1.0 - x)**2

        elif kindL == 'neumann' and kindR == 'dirichlet':
            # A(x) = q0*(x - x^2/2) + (β - q0/2)*x,  S(x) = x^2(1-x)
            A = valL*(x - x**2/2.0) + (valR - 0.5*valL)*x
            S = x**2*(1.0 - x)

        elif kindL == 'neumann' and kindR == 'neumann':
            # A'(0)=q0, A'(1)=q1；额外把常数规约 C=0 (否则解欠定)
            # A(x) = q0*(x - x^2/2) + q1*(x^2/2),  S(x)=x^2(1-x)^2  (使 S'(0)=S'(1)=0)
            A = valL*(x - x**2/2.0) + valR*(x**2/2.0)
            S = (x**2)*((1.0 - x)**2)

        else:
            raise ValueError("BC must be combinations of 'dirichlet' or 'neumann'.")

        return A, S

    def u_of_x(x):
        A, S = A_and_S(x)
        return A + S * model(x)

    return u_of_x


### Collocation Sampling

We draw collocation points $x_i\sim \mathcal{U}(0,1)$ for enforcing the PDE residual in the interior.

In [ ]:
# ==== Collocation sampling (collocation sampling)====
def sample_collocation(ncol):
    # uniform random sampling on (0,1)
    return torch.rand(ncol, 1, device=DEVICE, dtype=torch.float64)


### Training for a Single $\varepsilon$

Trains the model for a given $\varepsilon$ by minimizing a weighted sum of the interior (PDE) loss and boundary (BC) loss:
$$ \mathcal{L} = \lambda_{\text{PDE}}\,\big\| r(x) \big\|_2^2 \; + \; \lambda_{\text{BC}}\,\big\| g(\text{BC}) \big\|_2^2, $$
where $r(x)$ is the PDE residual and $g(\text{BC})$ encodes boundary mismatches (or is zero if BCs are hard-encoded).
Returns sampled $x$, predicted $u_\theta(x)$, (optional) exact solution, and the training history.

In [ ]:
# ==== Train one ε ====
def train_one_eps(
    eps, *, bc_left, bc_right, use_hard_bc,
    ncol, nedge, epochs, lr, print_every, lam_bc, lam_pde,
    seed=42
):
    torch.manual_seed(seed); np.random.seed(seed)

    model = MLP(width=64, depth=4).to(DEVICE)
    optim = torch.optim.Adam(model.parameters(), lr=lr)

    # 选择 u(x) 的实现：hard-encoding or 软约束
    if use_hard_bc:
        u_fn = build_hard_u(model, bc_left, bc_right)
        bc_loss_fn = (lambda: torch.tensor(0.0, device=DEVICE, dtype=torch.float64))  # hard-encoding无需边界损失
    else:
        u_fn = lambda x: model(x)
        bc_loss_fn = (lambda: boundary_loss(u_fn, nedge, bc_left, bc_right))

    hist = {'loss_pde': [], 'loss_bc': [], 'loss_total': []}

    for ep in range(1, epochs + 1):
        model.train()
        x_col = sample_collocation(ncol)

        # PDE residual
        res = pde_residual(u_fn, x_col, eps)
        loss_pde = torch.mean(res**2)

        # 边界损失 (hard-encoding时为 0)
        loss_bc = bc_loss_fn()

        loss = lam_pde * loss_pde + lam_bc * loss_bc

        optim.zero_grad()
        loss.backward()
        optim.step()

        if ep % print_every == 0:
            print(f"[eps={eps:.2e}] epoch={ep:5d}  "
                  f"loss_pde={loss_pde.item():.3e}  loss_bc={loss_bc.item():.3e}  total={loss.item():.3e}")

        hist['loss_pde'].append(loss_pde.item())
        hist['loss_bc'].append(loss_bc.item())
        hist['loss_total'].append(loss.item())

    # 推理/可视化数据
    model.eval()
    with torch.no_grad():
        x_plot = torch.linspace(0, 1, 400, device=DEVICE, dtype=torch.float64).view(-1,1)
        u_pred = u_fn(x_plot).cpu().numpy().ravel()
    x_plot_np = x_plot.cpu().numpy().ravel()

    # 若两端是 Dirichlet，给出解析解对比
    u_ex = None
    
    u_ex = exact_solution_dirichlet_dirichlet(x_plot_np, eps, alpha=float(bc_left[1]), beta=float(bc_right[1]))

    return x_plot_np, u_pred, u_ex, hist


### Sweep Over a List of $\varepsilon$ Values

Runs training sequentially for each $\varepsilon$ in a list (a simple homotopy/continuation strategy), collecting the curves for later visualization and comparison.

In [ ]:
# ==== Helper to run a list of eps ====
def run_all_eps(
    eps_list, *, bc_left, bc_right, use_hard_bc,
    ncol, nedge, epochs, lr, print_every, lam_bc, lam_pde
):
    curves = []
    for eps in eps_list:
        x_np, u_pred, u_ex, hist = train_one_eps(
            eps,
            bc_left=bc_left, bc_right=bc_right, use_hard_bc=use_hard_bc,
            ncol=ncol, nedge=nedge, epochs=epochs, lr=lr,
            print_every=print_every, lam_bc=lam_bc, lam_pde=lam_pde
        )
        curves.append((eps, x_np, u_pred, u_ex))
    return curves

def plot_curves(curves, title_suffix=""):
    plt.figure(figsize=(7.2, 4.6))
    for eps, x_np, u_pred, u_ex in curves:
        plt.plot(x_np, u_pred, '-', label=f'PINN pred, eps={eps:g}')
        if u_ex is not None:
            plt.plot(x_np, u_ex, '--', label=f'exact, eps={eps:g}')
    plt.xlabel('x'); plt.ylabel('u(x)')
    plt.title(f"PINN for -ε u'' + b u' + c u = f on (0,1) {title_suffix}")
    plt.legend(); plt.tight_layout(); plt.show()


### Results Table Utility

Builds a common grid, interpolates predicted and exact solutions, and computes absolute and relative errors for easy comparison across different $\varepsilon$.

In [ ]:
# ==== Results table utility (skip if you already added it)====
def build_results_table(curves, grid_n: int = 201):
    x_common = np.linspace(0.0, 1.0, grid_n)
    data = {'x': x_common}
    metrics_rows = []
    for eps, x_np, u_pred, u_ex in curves:
        u_pred_i = np.interp(x_common, x_np, u_pred)
        data[f'pred[eps={eps:g}]'] = u_pred_i
        if u_ex is not None:
            u_ex_i = np.interp(x_common, x_np, u_ex)
            data[f'exact[eps={eps:g}]'] = u_ex_i
            abs_err = np.abs(u_pred_i - u_ex_i)
            rel_err = abs_err / (np.maximum(np.abs(u_ex_i), 1e-12))
            data[f'abs_err[eps={eps:g}]'] = abs_err
            data[f'rel_err%[eps={eps:g}]'] = 100.0 * rel_err
            l2_rel = np.linalg.norm(u_pred_i - u_ex_i) / (np.linalg.norm(u_ex_i) + 1e-12)
            linf_rel = np.max(rel_err)
            metrics_rows.append({
                'eps': eps,
                'L2_rel': float(l2_rel),
                'Linf_rel': float(linf_rel),
                'MAE': float(np.mean(abs_err)),
                'RMSE': float(np.sqrt(np.mean(abs_err**2)))
            })
        else:
            data[f'exact[eps={eps:g}]'] = np.nan
            data[f'abs_err[eps={eps:g}]'] = np.nan
            data[f'rel_err%[eps={eps:g}]'] = np.nan
    df_table = pd.DataFrame(data)
    df_metrics = pd.DataFrame(metrics_rows)
    return df_table, df_metrics

def show_and_save_tables(df_table, df_metrics,
                         table_path='pinn_results_table.csv',
                         metrics_path='pinn_results_metrics.csv'):
    from IPython.display import display
    display(df_table)
    if not df_metrics.empty:
        print("\n—— 指标汇总 (metrics)——")
        display(df_metrics)
    df_table.to_csv(table_path, index=False)
    if not df_metrics.empty:
        df_metrics.to_csv(metrics_path, index=False)
    print(f"\n已保存: {table_path}")
    if not df_metrics.empty:
        print(f"已保存: {metrics_path}")


### Integrated Training Entry Point: `train(...)`

A convenience wrapper that wires together the components above using global hyperparameters (sampling counts, epochs, learning rate, logging cadence, loss weights, and whether BCs are hard-encoded).
It orchestrates the full training loop over a specified list of $\varepsilon$ values and produces plots and/or tables.

In [ ]:
# ==== Integrated training API train(...) ====
from typing import List, Tuple, Optional
import os
import pandas as pd

def train(EPS_LIST: List[float],
          layer_side: str = 'right',
          bc_left: Tuple[str, float] = ('dirichlet', 0.0),
          bc_right: Tuple[str, float] = ('dirichlet', 1.0),
          seed: int = 42):
    """
    Unified training entry point (Integrated training entry-point)
    - Uses global hyperparameters：NCOL, NEDGE, EPOCHS_PER_STAGE, LR, PRINT_EVERY, PLOT_EVERY,
                    LAM_BC, lam_pde, USE_HARD_BC
    - 训练每个 eps，按 PRINT_EVERY 打印；按 PLOT_EVERY 进行中途可视化 (可选)。
    - 结束时：
        1) 画一张包含所有 eps 的整合图，并保存为 'pinn_all_eps.png'
        2) 生成“结果汇总表格”和“指标表”，保存为 CSV

    返回:
        curves: List[(eps, x_np, u_pred, u_ex or None)]
        df_table: 逐点汇总表 (预测/真解/误差列)
        df_metrics: 指标汇总表
        fig_path: 最终整合图的文件路径
    """
    # 读取全局超参数
    global NCOL, NEDGE, EPOCHS_PER_STAGE, LR, PRINT_EVERY, PLOT_EVERY
    global LAM_BC, lam_pde, USE_HARD_BC

    torch.manual_seed(seed); np.random.seed(seed)

    curves = []
    # 逐个 eps 训练
    for eps in EPS_LIST:
        # 构建模型与优化器
        model = MLP(width=64, depth=4).to(DEVICE)
        optim = torch.optim.Adam(model.parameters(), lr=LR)

        # u(x) 选择：hard-encoding or 软约束
        if USE_HARD_BC:
            u_fn = build_hard_u(model, bc_left, bc_right)
            def bc_loss_fn():
                return torch.tensor(0.0, device=DEVICE, dtype=torch.float64)
        else:
            u_fn = lambda x: model(x)
            def bc_loss_fn():
                return boundary_loss(u_fn, NEDGE, bc_left, bc_right)

        # 训练循环
        for ep in range(1, EPOCHS_PER_STAGE + 1):
            model.train()
            x_col = sample_collocation(NCOL)

            # PDE residual损失 (residual loss)
            res = pde_residual(u_fn, x_col, eps)
            loss_pde = torch.mean(res**2)

            # 边界损失 (hard-encoding为 0)
            loss_bc = bc_loss_fn()

            loss = lam_pde * loss_pde + LAM_BC * loss_bc

            optim.zero_grad()
            loss.backward()
            optim.step()

            # 打印 (PRINT_EVERY)
            if ep % PRINT_EVERY == 0:
                print(f"[eps={eps:.2e}] epoch={ep:5d}  "
                      f"loss_pde={loss_pde.item():.3e}  "
                      f"loss_bc={loss_bc.item():.3e}  total={loss.item():.3e}")

            # 中途可视化 (PLOT_EVERY)，可设 PLOT_EVERY<=0 以关闭
            if PLOT_EVERY and PLOT_EVERY > 0 and (ep % PLOT_EVERY == 0 or ep == EPOCHS_PER_STAGE):
                model.eval()
                with torch.no_grad():
                    x_plot = torch.linspace(0, 1, 400, device=DEVICE, dtype=torch.float64).view(-1,1)
                    u_pred = u_fn(x_plot).cpu().numpy().ravel()
                x_plot_np = x_plot.cpu().numpy().ravel()

                u_ex = None
                
                u_ex = exact_solution(
                        x_plot_np, eps
                    )

                # 单 eps 小图 (不保存，只显示)
                plt.figure(figsize=(6.4, 3.8))
                plt.plot(x_plot_np, u_pred, '-', label=f'PINN pred (eps={eps:g}, ep={ep})')
                if u_ex is not None:
                    plt.plot(x_plot_np, u_ex, '--', label='exact')
                plt.xlabel('x'); plt.ylabel('u(x)')
                title_mode = 'hard' if USE_HARD_BC else 'soft'
                plt.title(f'PINN ({title_mode})  eps={eps:g}  epoch={ep}')
                plt.legend(); plt.tight_layout(); plt.show()

        # 该 eps 的最终曲线 (用于整合图与表格)
        model.eval()
        with torch.no_grad():
            x_plot = torch.linspace(0, 1, 400, device=DEVICE, dtype=torch.float64).view(-1,1)
            u_pred = u_fn(x_plot).cpu().numpy().ravel()
        x_plot_np = x_plot.cpu().numpy().ravel()

        u_ex = None
        if True:
            u_ex = exact_solution(
                x_plot_np, eps
            )

        curves.append((eps, x_plot_np, u_pred, u_ex))

    # ======= 训练完成：整合图 =======
   
    plt.figure(figsize=(7.6, 4.8))
    for eps, x_np, u_pred, u_ex in curves:
        plt.plot(x_np, u_pred, '-', label=f'pred, eps={eps:g}')
        if u_ex is not None:
            plt.plot(x_np, u_ex, '--', label=f'exact, eps={eps:g}')
    title_mode = 'hard' if USE_HARD_BC else 'soft'
    plt.xlabel('x'); plt.ylabel('u(x)')
    plt.title(f"PINN ({title_mode})  layer_side={layer_side}  on (0,1)")
    plt.legend(); plt.tight_layout()
    plt.show()
    




In [ ]:
# =========================== Hyperparameters (last cell) ===========================
# Manually specify boundary-layer side and boundary conditions
LAYER_SIDE = 'right'                  # 'left' or 'right'，set manually after inspection (unused in this implementation)
BC_LEFT  = ('dirichlet', 0.0)        # ('dirichlet', α) or ('neumann', q0)
BC_RIGHT = ('dirichlet', 1.0)        # ('dirichlet', β) or ('neumann', q1)

# Training-related
EPS_LIST = [ 1e-2, 5e-3, 1e-3, 1e-4]                  # e.g., could be changed to [5e-2, 1e-2, 5e-3, 1e-3]
NCOL, NEDGE = 2048, 512
EPOCHS_PER_STAGE = 2000
PRINT_EVERY, PLOT_EVERY = 100, 500
LR = 1e-3

# Constraint mode (kept for hyperparameter compatibility)：'hard' or 'soft' (本实现不依赖此开关，但保持一致性)
BC_MODE = 'hard'
USE_HARD_BC = BC_MODE == 'hard'
LAM_BC = 1.0                         # 软约束下的边界损失权重 λ_bc
# 下面这些在当前“最传统 PINN”里不用，但按你的要求保留
S0, S1, kappa = 5.0, 7.0, 2.0
lam_comp = 0.05
tau, eta_phi = 0.1, 1e-4
lam_pde, lam_phi = 1.0, 0.01        # 这里我们会用到 lam_pde (PDE residual权重)


# ======================================================================


In [ ]:

def b_fun(x): return torch.ones_like(x)
def c_fun(x): return torch.zeros_like(x)
def f_fun(x): return torch.exp(x)

def exact_solution(x_np, eps):
    import numpy as np
    x = np.asarray(x_np, dtype=np.float64)
    a = 1.0/eps

    # stable ratio R(x) in [0,1]
    # R(x) = (e^{a x}-1)/(e^{a}-1) = e^{a(x-1)} * (1 - e^{-a x})/(1 - e^{-a})
    exp_neg_a = np.exp(-a)
    numerator = np.exp(a*(x-1.0)) * (1.0 - np.exp(-a*x))
    denominator = 1.0 - exp_neg_a
    R = numerator / denominator

    # compact/stable closed form:
    # u(x) = [ e^{x} - ε + (ε - e) * R(x) ] / (1 - ε)
    u = (np.exp(x) - eps + (eps - np.e) * R) / (1.0 - eps)
    return u.astype(np.float64)
    

# Example B: left boundary layer
LAYER_SIDE = 'left'; BC_LEFT = ('dirichlet', 1.0); BC_RIGHT = ('dirichlet', 0.0)
EPS_LIST = [5e-2, 1e-2, 5e-3, 1e-3]
train(EPS_LIST, layer_side=LAYER_SIDE, bc_left=BC_LEFT, bc_right=BC_RIGHT)

In [ ]:

train(
    EPS_LIST,
    layer_side=LAYER_SIDE,
    bc_left=BC_LEFT,
    bc_right=BC_RIGHT
)

In [ ]:
# PDE:  ε y'' + y' + 2e^{-x} = 0   ==>  -ε y'' - y' - 2e^{-x} = 0
def b_fun(x): return -torch.ones_like(x)        # b = -1
def c_fun(x): return torch.zeros_like(x)        # c = 0
def f_fun(x): return 2.0 * torch.exp(-x)        # f = 2 e^{-x}

def exact_solution(x_np, eps):
    x = np.asarray(x_np, dtype=np.float64)
    return 2.0*np.exp(-x) - 2.0*eps*np.exp(-x/eps)

# Left boundary layer (b<0)
LAYER_SIDE = 'left'

# Left Neumann (q0=0), Right Dirichlet (β=2e^{-1})
BC_LEFT  = ('neumann', 0.0)
BC_RIGHT = ('dirichlet', float(2.0*np.exp(-1.0)))

# Your homotopy parameters
EPS_LIST = [5e-4, 1e-4, 5e-5, 1e-5]
train(EPS_LIST, layer_side=LAYER_SIDE, bc_left=BC_LEFT, bc_right=BC_RIGHT)